In [18]:
def deferred_acceptance(prop_prefs, resp_prefs,caps=None): 
    indptr=[0,1,3,4]#indptrの導出方法はまだ分からず。
    import numpy as np
    cp_resp_prefs=[]
    for i in range(len(resp_prefs)):
        a= list(resp_prefs[i])
        for j in range(caps[i]):
            cp_resp_prefs.append(a)#大学側の選好を複製
    
    for j in range(len(prop_prefs)):
        a=prop_prefs[j]
        for i in range(len(indptr)-1,0,-1):
            a[a.index(i)]=indptr[i]
        indptr.pop(-1)    
        for i in indptr:
            a[a.index(i):a.index(i)+1]=(np.array([i]*caps[indptr.index(i)]))+(np.array(range(caps[indptr.index(i)])))
        prop_prefs[j]=a
        indptr.append(sum(caps))#学生側の選好を複製した大学の番号に合わせる

    pnum = len(prop_prefs) #pnumをunmatchedにも対応させている。 prop_prefs :  m  x  (n+1)  の2次元配列
    rnum = len(cp_resp_prefs) # ここ要注意。
    free=range(pnum)  #受験者に番号をそれぞれ割り振る。最初はみんな内定とってない。
    omedeto = {}#合格者の組
    have = [pnum for i in range(rnum)]#最初は皆受け入れるので、付き合っている人は最も序列の低いunmatchedとしてのmnumで統一して全部入れる 
    down = [0 for i in range(pnum)]#０で全部入れる。 
    while len(free) > 0:

        for i in free:

            pbest = prop_prefs[i][down[i]]#iからアプローチされた大学

            if pbest != rnum:# アプローチされた大学が実態のある人なら
                if cp_resp_prefs[pbest].index(i) < cp_resp_prefs[pbest].index(have[pbest]): #aList = [123, 'xyz', 'zara', 'abc'でaList.index( 'xyz' ) は１、aList.index( 'zara' ) は２が返される。                     
#iが、アプローチされた女性の選好において何番目に位置するか＜すでに付き合っている男性（最初はunmatched)が、プローチされた女性の選好において何番目に位置するか                   
                    free.remove(i)#仮として結ばれ独身でなくなる 
                    have[pbest] = i#代わってiさんが仮の付き合ってる人になる 
                    if have[pbest] != pnum: #すでに付き合っている男性がunmatchedでなく、普通の男性なら、                          
                        free.append(have[pbest]) #その男性は独身になる 
                                    
                else: #iが、アプローチされた女性の選好において何番目に位置するか＞すでに付き合っている男性が、プローチされた女性の選好において何番目に位置するか　で恋に敗れてしまったら 
                    if down[i] < rnum - 1: #まだ希望があれば                          
                        down[i] += 1#選好を一つ落としてまた頑張ろう 
                    else:#もう選択肢がなければ、 
                        free.remove(i)#戦闘から離脱して大人しく、i: rnumってことにしろ                         
                        omedeto.update({i: rnum}) 
            else:#もうunmatchedだったら、そこで終わり。戦闘から離脱して大人しく、i: rnumってことにしろ 
                free.remove(i) 
                omedeto.update({i: rnum}) 
 
    for i in have: 
        if i != pnum: 
            omedeto.update({i: have.index(i)})#男各々iに対して順番に最終的に付き合っていた女性と結婚させる{0:2,1:18,2:28,3:14,}みたいに 
 
 
    omedeto = omedeto.items()#The method items() returns a list of dict's (key, value) tuple pairs 
                              #例えば、dict = {'Name': 'Zara', 'Age': 7}で、dict.items()とすると[('Age', 7), ('Name', 'Zara')]と返す 
                              #ここでは、[(0,2),(1,18),(2,28),(3,14)]みたいになる 
    prop_matched_computed = [] 
    for i in range(pnum): 
        prop_matched_computed.append(omedeto[i][1])#女性の番号を割り出す。 
    resp_matched_computed = have 
      
    return prop_matched_computed, resp_matched_computed 
d=deferred_acceptance([[0,1,2,3,],[1,0,2,3],[1,2,0,3],[0,2,1,3]],[[0,1,2,3],[0,2,1,3],[0,2,3,1]],[1,2,1] )
d

ValueError: 4 is not in list